In [1]:
from fastai.vision.all import *
from modules_th.video_block import *
from pathlib import Path
import pandas as pd
import os

In [2]:
#export
class DF2Paths(Transform): 
    def __init__(self, path, fps=24, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.path, self.fps = path, fps
        
    
    def encodes(self, df:pd.Series):
        def fr(t): return int(float(t)*self.fps)
        Id, start, end = df['id'], fr(df['start']) + 1, fr(df['end'])
        step = -1 if start > end else 1                     # If start is greater than end,
                                                            # it reverses the order of the for loop
                                                            # This because it seems some videos are in reverse
                
        def fr_path(n:int): return str(self.path/'Charades_v1_rgb'/Id/f'{Id}-{n:0>6d}.jpg')
        vid = [fr_path(n) for n in range(start, end, step) if os.path.exists(fr_path(n))]
        return vid

In [3]:
path_charades = Path('/mnt/data/adrianlopez/Videos/Charades') 
items = pd.read_csv(path_charades/'df0.csv', index_col=0)
items = uniformize_dataset(items.loc[items['test']==False])
items.head()

,id,coded_lbl,lbl,start,end,test
46344,G70CT,c015,Holding a phone/camera,20.5,32.0,False
25229,UJXBC,c015,Holding a phone/camera,17.3,27.0,False
18234,TSQBQ,c015,Holding a phone/camera,0.0,30.0,False
17416,IZTHW,c018,Taking a phone/camera from somewhere,25.6,34.0,False
38416,82B2V,c018,Taking a phone/camera from somewhere,9.0,17.0,False


In [4]:
item = items.iloc[43218]
pl = Pipeline([DF2Paths(path_charades, fps=24), Video.create, Resize(224), ResizeTime(), ToTensor()])
vid = pl(item)
vid.shape

torch.Size([50, 3, 224, 224])

In [5]:
def get_labels(df): return df['lbl']
splits = RandomSplitter(seed=42)(items)
ds = Datasets(items, [pl, [get_labels, Categorize()]], splits=splits)

In [6]:
dls = ds.dataloaders(bs=15, shuffle_fn=UniformizedShuffle(items.iloc[splits[0]]))
b = dls.one_batch()
b[0].shape

torch.Size([15, 50, 3, 224, 224])

In [7]:
b[1]

TensorCategory([121, 121, 121,  61,  61,  61,  86,  86,  86,  77,  77,  77,  57,  57,
         57], device='cuda:0')

In [8]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_datasets.ipynb.
Converted index.ipynb.


In [ ]:
# default_exp video_block